In [1]:
import pandas as pd
import numpy as np
import http.client, urllib.request, urllib.parse
import json
import os
from datetime import datetime, timedelta

In [2]:
#leagues = {'ONS': 720025558011953152, 'DODN': 786404550704758784}


## Run this cell and input your most recent league ID.
### The following script works backward to get each previous league/season.

league_id = input("League ID:")

League ID: 720025558011953152


In [3]:
leagues = list()
league_info = pd.DataFrame()
league_stats = pd.DataFrame()
users = pd.DataFrame()
rosters = pd.DataFrame()

conn = http.client.HTTPSConnection('api.sleeper.app')

while league_id:
    print('League: ' + str(league_id))
    leagues.append(league_id)
    conn.request("GET", '/v1/league/' + str(league_id))
    r = conn.getresponse().read()
    data = json.loads(r)
    season = data.get('season')
    weeks = int(data.get('settings').get('playoff_week_start')) - 1
    status = data.get('status')
    row = [league_id, season, status, weeks]
    league_info = league_info.append([row])
    
    
    
    if status == 'complete':
        for w in range(weeks):
            week = w + 1
            #print('---- Week: ' + str(week))
            
            endpoint = '/v1/league/' + str(league_id) + '/matchups/' + str(week)
            conn.request("GET", endpoint)
            m = conn.getresponse().read()
            mdata = json.loads(m)
            
            for tdata in mdata:
                try:
                    points = tdata.get('players_points')
                    roster_id = tdata.get('roster_id')
                    matchup = tdata.get('matchup_id')
                    players = pd.DataFrame({'roster_id': roster_id, 'season': season, 'week': week, 'matchup':matchup, 'player_id' : points.keys(), 'points': points.values()})
                    starters = tdata.get('starters')

                    players['starter'] = np.where(players['player_id'].isin(starters), True, False)
                    players['league_id'] = league_id

                    league_stats = league_stats.append(players)
                except:
                    #print('-------- Error for roster ' + str(roster))
                    continue
                    
        endpoint = '/v1/league/' + str(league_id) + '/users/'
        conn.request("GET", endpoint)
        u = conn.getresponse().read()
        udata = json.loads(u)
        for u in udata:
            user_id = u.get('user_id')
            display_name = u.get('display_name')
            urow = [user_id, display_name]
            users = users.append([urow])
            
        endpoint = '/v1/league/' + str(league_id) + '/rosters/'
        conn.request("GET", endpoint)
        b = conn.getresponse().read()
        bdata = json.loads(b)
        for b in bdata:
            roster_id = b.get('roster_id')
            user_id = b.get('owner_id')
            division = b.get('settings').get('division')
            brow = [roster_id, user_id, league_id, division]
            rosters = rosters.append([brow])


        
    
    league_id = data.get('previous_league_id')

users.columns = ['user_id','display_name']
rosters.columns = ['roster_id','user_id', 'league_id', 'division']



League: 720025558011953152
League: 599742529973305344
League: 384374154654392320
League: 342901160002236416


In [4]:
league_stats = league_stats.merge(rosters, how='left', on=['roster_id','league_id'])
league_stats = league_stats.merge(users, how='left', on='user_id')

In [5]:
league_stats = league_stats.drop_duplicates(subset=['league_id','user_id','season','week', 'player_id']).reset_index(drop=True)

In [16]:
# This function creates players.csv if it doesn't already exist.
# If players.csv does exist, it will update the file once a day.

def update_players(path = 'players.csv', manual = False, status = 'all'):
    today = datetime.today().date()
    
    try:
        t = os.path.getmtime(path)
        lu = datetime.fromtimestamp(t).date()
    except:
        lu = today - timedelta(days = 1)

    if lu < today:
        print('Updating players from API...')
        conn = http.client.HTTPSConnection('api.sleeper.app')
        conn.request("GET", "/v1/players/nfl")
        response = conn.getresponse().read()

        df = pd.read_json(response)
        df = df.transpose()
        df.to_csv('players.csv', header=True, index=False)
    else:
        print('Players already up to date.')
        df = pd.read_csv('players.csv')
        
    return df

In [17]:
players = update_players()


Updating players from API...


In [10]:
league_stats = league_stats.merge(players[['player_id','full_name', 'position']], how='left')
league_stats['full_name'] = np.where(league_stats['position'] == 'DEF', league_stats['player_id'],league_stats['full_name'])

In [11]:
league_stats.to_csv('league_stats.csv', index=False)

In [12]:
summary_data = league_stats[league_stats['starter'] == True]

summary = summary_data.groupby(['season', 'league_id', 'week','roster_id','matchup'])['points'].sum().reset_index()

# get results for each matchup
matchup = summary.groupby(['season','week','matchup'])['points'].max().reset_index()
summary = summary.merge(matchup, how='left', on=['season','week','matchup'])
summary['result'] = np.where(summary['points_x'] == summary['points_y'], 1, 0)
summary = summary.drop('points_y', axis = 1)
summary.columns = ['season', 'league_id', 'week', 'roster_id', 'matchup', 'points', 'result']
del matchup

# which teams beat the median each week?
median = summary.groupby(['season','week'])['points'].median().reset_index()
summary = summary.merge(median, how='left', on=['season','week'])
summary['top_half'] = np.where(summary['points_x'] > summary['points_y'], 1, 0)
summary = summary.drop('points_y', axis = 1)
summary.columns = ['season', 'league_id', 'week', 'roster_id', 'matchup', 'points', 'result', 'median']

del median

# best score each week
maximum = summary.groupby(['season','week'])['points'].max().reset_index()
summary = summary.merge(maximum, how='left', on=['season','week'])
summary['top_half'] = np.where(summary['points_x'] == summary['points_y'], 1, 0)
summary = summary.drop('points_y', axis = 1)
summary.columns = ['season', 'league_id', 'week', 'roster_id', 'matchup', 'points', 'result', 'median', 'best_score']
del maximum

# worst score each week
minimum = summary.groupby(['season','week'])['points'].min().reset_index()
summary = summary.merge(minimum, how='left', on=['season','week'])
summary['top_half'] = np.where(summary['points_x'] == summary['points_y'], 1, 0)
summary = summary.drop('points_y', axis = 1)
summary.columns = ['season', 'league_id', 'week', 'roster_id', 'matchup', 'points', 'result', 'median', 'best_score', 'worst_score']
del minimum

summary = summary.merge(rosters, how='left', on=['roster_id','league_id'])
summary = summary.merge(users, how='left', on='user_id')
summary = summary.drop_duplicates(subset=['user_id','league_id','season','week'])

In [13]:
summary.to_csv('results.csv', index=False)